## 核心概念讲解

### 1. **ChatPromptTemplate基础结构**
```python
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", system_template),
    ("human", human_template)
])
```
- `system`: 设定AI助手的角色和行为规范
- `human`: 用户的输入消息
- 支持变量插值，使用`{variable_name}`语法

### 2. **LCEL管道操作符 `|`**
```python
chain = chat_prompt | model | output_parser
```
这是LCEL的核心特性：
- `|` 操作符将多个组件链接成流水线
- 数据从左到右流动
- 每个组件的输出成为下一个组件的输入

### 3. **关键组件说明**

**ChatPromptTemplate**: 
- 构建结构化的对话提示
- 支持多种消息类型（system, human, assistant）
- 可以包含变量占位符

**Model (ChatOpenAI)**:
- 实际的语言模型
- 接收格式化的提示，返回AI响应

**StrOutputParser**:
- 将模型输出解析为字符串
- 简化输出格式

### 4. **高级特性**

**RunnablePassthrough**:
```python
RunnablePassthrough.assign(processed_data=process_user_data)
```
- 允许在链中插入数据处理步骤
- `assign`方法可以添加新的数据字段

**占位符 (placeholder)**:
```python
("placeholder", "{chat_history}")
```
- 用于插入动态内容，如历史对话
- 支持复杂的对话上下文管理

### 5. **LCEL的优势**

1. **可读性**: 管道操作符让数据流向清晰可见
2. **模块化**: 每个组件职责单一，易于测试和维护
3. **可组合性**: 可以轻松重组和扩展链
4. **错误处理**: 支持内置的错误处理和回退机制
5. **批处理**: 自动支持批量处理请求

### 6. **实际使用建议**

1. **模板设计**: 系统消息要明确角色定位和回答要求
2. **变量命名**: 使用有意义的变量名提高可读性
3. **错误处理**: 在生产环境中添加异常捕获
4. **性能优化**: 对于频繁调用的链考虑缓存机制

这种LCEL风格的写法让LangChain应用的构建变得更加直观和强大，特别适合构建复杂的AI应用流水线。

In [1]:
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough

# ================================
# 示例1: 基础的ChatPromptTemplate使用
# ================================

# 创建系统消息模板
system_template = """你是一个专业的{role}。请根据用户的问题，
提供准确、详细的回答。回答应该具有以下特点：
- 专业性强
- 逻辑清晰 
- 易于理解
"""

# 创建人类消息模板
human_template = "请回答关于{topic}的问题：{question}"

# 构建ChatPromptTemplate
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", system_template),
    ("human", human_template)
])

# 初始化模型和输出解析器
model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)
output_parser = StrOutputParser()

# LCEL风格的链式调用
basic_chain = chat_prompt | model | output_parser

# 使用示例
def run_basic_example():
    result = basic_chain.invoke({
        "role": "Python编程专家",
        "topic": "机器学习",
        "question": "什么是梯度下降算法？"
    })
    print("=== 基础示例结果 ===")
    print(result)
    print()

# ================================
# 示例2: 更复杂的多轮对话模板
# ================================

# 创建包含历史对话的模板
conversation_template = ChatPromptTemplate.from_messages([
    ("system", """你是一个智能助手，名字叫{assistant_name}。
    你的特点是：{personality}
    
    请基于以下对话历史，继续对话："""),
    ("placeholder", "{chat_history}"),  # 用于插入历史对话
    ("human", "{user_input}")
])

# 带有历史记录的链
conversation_chain = conversation_template | model | output_parser

def run_conversation_example():
    # 模拟历史对话
    chat_history = [
        ("human", "你好，请介绍一下自己"),
        ("assistant", "你好！我是小智，一个友好的AI助手。我很乐意帮助你解答问题！"),
        ("human", "你能帮我学习编程吗？"),
        ("assistant", "当然可以！我很擅长编程教学，你想学习哪种编程语言呢？")
    ]
    
    result = conversation_chain.invoke({
        "assistant_name": "小智",
        "personality": "友好、耐心、专业",
        "chat_history": chat_history,
        "user_input": "我想学习Python，从哪里开始比较好？"
    })
    
    print("=== 对话示例结果 ===")
    print(result)
    print()

# ================================
# 示例3: 结合RunnablePassthrough的高级用法
# ================================

# 创建一个数据处理函数
def process_user_data(inputs):
    """处理用户输入数据"""
    user_info = inputs["user_info"]
    question = inputs["question"]
    
    # 提取用户信息
    name = user_info.get("name", "用户")
    experience = user_info.get("experience", "初学者")
    interests = user_info.get("interests", [])
    
    return {
        "name": name,
        "experience": experience,
        "interests": ", ".join(interests) if interests else "通用知识",
        "question": question
    }

# 个性化回答模板
personalized_template = ChatPromptTemplate.from_messages([
    ("system", """你是一个个性化学习助手。请根据用户的背景信息提供定制化回答：

用户姓名：{name}
经验水平：{experience}
兴趣领域：{interests}

请根据用户的背景，调整回答的深度和风格。对于初学者使用简单易懂的语言，
对于有经验的用户可以提供更深入的技术细节。"""),
    ("human", "{question}")
])

# 构建包含数据处理的完整链
personalized_chain = (
    RunnablePassthrough.assign(processed_data=process_user_data) |
    (lambda x: x["processed_data"]) |
    personalized_template | 
    model | 
    output_parser
)

def run_personalized_example():
    user_data = {
        "user_info": {
            "name": "张三",
            "experience": "中级开发者",
            "interests": ["机器学习", "深度学习", "自然语言处理"]
        },
        "question": "如何选择合适的神经网络架构？"
    }
    
    result = personalized_chain.invoke(user_data)
    
    print("=== 个性化示例结果 ===")
    print(result)
    print()

# ================================
# 示例4: 条件逻辑和动态模板选择
# ================================

def select_template(inputs):
    """根据问题类型选择不同的模板"""
    question_type = inputs.get("type", "general")
    
    templates = {
        "technical": ChatPromptTemplate.from_messages([
            ("system", "你是一个技术专家。请提供详细的技术解答，包括代码示例。"),
            ("human", "技术问题：{question}")
        ]),
        "creative": ChatPromptTemplate.from_messages([
            ("system", "你是一个创意写作助手。请发挥想象力，提供有趣的创意回答。"),
            ("human", "创意问题：{question}")
        ]),
        "general": ChatPromptTemplate.from_messages([
            ("system", "你是一个通用助手。请提供准确、有用的回答。"),
            ("human", "问题：{question}")
        ])
    }
    
    return templates.get(question_type, templates["general"])

# 动态模板选择链
def create_dynamic_chain(question_type):
    template = select_template({"type": question_type})
    return template | model | output_parser

def run_dynamic_example():
    # 技术问题
    tech_chain = create_dynamic_chain("technical")
    tech_result = tech_chain.invoke({
        "question": "如何在Python中实现单例模式？"
    })
    
    print("=== 技术问题结果 ===")
    print(tech_result)
    print()
    
    # 创意问题
    creative_chain = create_dynamic_chain("creative")
    creative_result = creative_chain.invoke({
        "question": "写一个关于AI和人类友谊的短故事开头"
    })
    
    print("=== 创意问题结果 ===")
    print(creative_result)
    print()

# ================================
# 示例5: 错误处理和回退机制
# ================================

from langchain_core.runnables import RunnableLambda

def safe_invoke_with_fallback(chain, inputs, fallback_message="抱歉，处理您的请求时出现了问题。"):
    """安全调用链，包含错误处理"""
    try:
        return chain.invoke(inputs)
    except Exception as e:
        print(f"错误: {e}")
        return fallback_message

# 带错误处理的链
safe_chain = RunnableLambda(
    lambda inputs: safe_invoke_with_fallback(basic_chain, inputs)
)

# ================================
# 主函数 - 运行所有示例
# ================================

def main():
    """运行所有示例"""
    print("LangChain ChatPromptTemplate LCEL风格示例")
    print("=" * 50)
    
    # 注意：实际运行需要设置OpenAI API密钥
    # import os
    # os.environ["OPENAI_API_KEY"] = "your-api-key-here"
    
    try:
        # 运行基础示例
        run_basic_example()
        
        # 运行对话示例
        run_conversation_example()
        
        # 运行个性化示例
        run_personalized_example()
        
        # 运行动态模板示例
        run_dynamic_example()
        
    except Exception as e:
        print(f"运行示例时出错: {e}")
        print("请确保已正确设置OpenAI API密钥")

if __name__ == "__main__":
    main()

# ================================
# 额外工具函数
# ================================

def print_chain_structure(chain):
    """打印链的结构（用于调试）"""
    print("链结构:")
    print(chain)
    print()

def batch_process_questions(chain, questions_list):
    """批量处理问题"""
    results = chain.batch(questions_list)
    for i, result in enumerate(results):
        print(f"问题 {i+1} 结果: {result}")
    return results

# 使用示例：
# questions = [
#     {"role": "数据科学家", "topic": "统计学", "question": "什么是p值？"},
#     {"role": "软件工程师", "topic": "算法", "question": "解释快速排序算法"}
# ]
# batch_process_questions(basic_chain, questions)

LangChain ChatPromptTemplate LCEL风格示例
=== 基础示例结果 ===
梯度下降算法是一种常用的优化算法，用于在机器学习中寻找最优模型参数。其基本思想是通过迭代的方式不断调整模型参数，使目标函数（损失函数）的值达到最小化。

具体而言，梯度下降算法通过计算目标函数对于参数的梯度（导数），然后沿着梯度的反方向更新参数，以实现不断降低目标函数值的目的。这样的迭代过程会持续进行直到达到某个终止条件，比如达到最大迭代次数、参数变化很小时等。

梯度下降算法主要分为批量梯度下降（Batch Gradient Descent）、随机梯度下降（Stochastic Gradient Descent）和小批量梯度下降（Mini-batch Gradient Descent）三种形式。其中，批量梯度下降会在每次迭代中使用所有样本来更新参数，随机梯度下降则是每次只使用一个样本，而小批量梯度下降则是在中间值进行权衡，同时使用一批样本。

梯度下降算法的优点包括易于实现、收敛速度快等，但也存在一些缺点，比如可能陷入局部最优解、对初始值敏感等。因此，在实际应用中，需要根据具体问题和数据集的特点选择合适的梯度下降算法及调参策略。

=== 对话示例结果 ===
学习Python是一个很好的选择！你可以从以下几个步骤开始：

1. 安装Python：首先，你需要安装Python解释器。你可以在Python官方网站上下载最新版本的Python并安装到你的电脑上。

2. 学习基础语法：学习Python的基础语法，了解变量、数据类型、条件语句、循环语句等基本概念。

3. 练习编写代码：通过练习编写简单的Python程序来巩固所学知识，比如编写一个简单的计算器程序或者输出特定格式的字符串。

4. 学习函数和模块：学习如何定义和调用函数，以及如何使用模块扩展Python的功能。

5. 深入学习：学习面向对象编程、异常处理、文件操作等进阶内容，掌握更多Python编程技巧。

如果有任何问题或者需要帮助，随时都可以问我哦！我会尽力帮助你学习Python编程。

=== 个性化示例结果 ===
对于中级开发者来说，选择合适的神经网络架构是非常重要的。在机器学习、深度学习和自然语言处理领域，有许多常用的神经网络架构可供选择，如卷积神经网络（CNN）、循环神经网络（RNN